In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db


# **Finding the real murder of the crimes by SQL**

in this notebook, we will demonstrate SQL query to inspect the crime data from SQL Murder data mystery database. We will become conan the detectives.📍

# Step 1 : Setting Environment and Connect to Database

In [2]:
# import some useful libraries
import sqlite3 as sql
import numpy as np
import pandas as np
pd.set_option('display.max_colwidth', None)

In [3]:
# coonect to the database
con = sql.connect('../input/sql-murder-mystery-database/sql-murder-mystery.db')

at this step, the database will connect to the notebook. We are ready to dive into data.

# Step 2 : Exploring the data

In [4]:
query1 = """
SELECT *
FROM crime_scene_report LIMIT 5
"""
pd.read_sql_query(query1, con)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


In [5]:
# the most frequent city in crime
query2 = """
SELECT city, count(city) AS number_of_crime from crime_scene_report
group by city
order by count(city) DESC;
"""
pd.read_sql_query(query2, con)

,city,number_of_crime
0,SQL City,9
1,Murfreesboro,9
2,Toledo,8
3,Jersey City,8
4,Evansville,8
...,...,...
372,Barnstable,1
373,Arvada,1
374,Allentown,1
375,Alexandria,1


the most frequent cities are SQL City and Murfreeboro

In [6]:
query3 = """
SELECT date, city, description from crime_scene_report
where city == "SQL City";
"""
pd.read_sql_query(query3, con)

,date,city,description
0,20180215,SQL City,REDACTED REDACTED REDACTED
1,20180215,SQL City,Someone killed the guard! He took an arrow to the knee!
2,20170712,SQL City,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hiding, no escape. In the distance, the beast\n falters, tethered to the void. The killing blow comes without\n hesitation, without mercy."
3,20170820,SQL City,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and future."
4,20171110,SQL City,"The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardians who fell at the Twilight Gap. Gifted\n to the survivors of that terrible battle, the Gjallarhorn\n is seen as a symbol of honor and survival."
5,20180103,SQL City,"Apparently, Cayde thought it necessary to expose this extremely\n rare vegetable to a Hive summoning ritual."
6,20180115,SQL City,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!"
7,20180115,SQL City,Report Not Found
8,20180115,SQL City,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave""."


In [7]:
query4 = """
SELECT city, description from crime_scene_report
where city == "Murfreesboro";
"""
pd.read_sql_query(query4, con)

,city,description
0,Murfreesboro,"her, to pass away the time.\n"
1,Murfreesboro,"‘An arm, you goose! Who ever saw one that size? Why, it fills the whole\n"
2,Murfreesboro,all talking together: she made out the words: ‘Where’s the other\n
3,Murfreesboro,"executed for having missed their turns, and she did not like the look\n"
4,Murfreesboro,"candle is like after the candle is blown out, for she could not remember\n"
5,Murfreesboro,was a little startled by seeing the Cheshire Cat sitting on a bough of a\n
6,Murfreesboro,\n
7,Murfreesboro,And concluded the banquet--]\n
8,Murfreesboro,"said Alice, in a coaxing tone, and she tried hard to whistle to it; but\n"


Since the description on Murfreesboro is all the way out of control, we will focus crimes happened in **SQL city.**

# Step 3 : Look at what witness said

From the clues, there was a witness near **Northwestern Dr** and **Franklin Ave** and he had an Arrow with him.

1st cuprit : last house of Northwestern Dr

In [8]:
query5 = """
SELECT * FROM person
where address_street_name = "Northwestern Dr" 
order by address_number DESC
LIMIT 1;
"""
pd.read_sql_query(query5, con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


2nd culprit : named Annabel, lives somewhere on "Franklin Ave"

In [9]:
query6 = """
SELECT *
FROM person
WHERE name like '%Annabel%' AND address_street_name = "Franklin Ave";
"""
pd.read_sql_query(query6, con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Those 2 witnesses are Annabel Miller (16731) and Morty Schapiro (14887). We are going to see what our witnesses said.

In [10]:
query8 = """
SELECT *
FROM interview
WHERE person_id in ("14887","16371");
"""
pd.read_sql_query(query8, con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


our culprit has 5 criterias
1. have a gun
2. There is a get fit now gym bag.
3. member num starts at 48Z
4. car has H42W in plate
5. Born on 9 January

# Step 4 : finding our culprit

Culprit was going to gym on January 9, 2018

In [11]:
query9 = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = "20180109";
"""
pd.read_sql_query(query9, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,X0643,20180109,957,1164
1,UK1F2,20180109,344,518
2,XTE42,20180109,486,1124
3,1AE2H,20180109,461,944
4,6LSTG,20180109,399,515
5,7MWHJ,20180109,273,885
6,GE5Q8,20180109,367,959
7,48Z7A,20180109,1600,1730
8,48Z55,20180109,1530,1700
9,90081,20180109,1600,1700


Aha, I saw person with membership_id 48Z7A , 48Z55. interesting.

In [12]:
query10 = """
SELECT *
FROM get_fit_now_member
WHERE id in ("48Z7A","48Z55");
"""
pd.read_sql_query(query10, con)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


In [13]:
query11 = """
SELECT *
FROM person
WHERE id in ("67318","28819");
"""
pd.read_sql_query(query11, con)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


We trace membership_id. We curious Joe Germuska and Jeremy Bowers. 

In [14]:
query12 = """
SELECT *
FROM drivers_license
WHERE id in ("178289","423327");
"""
pd.read_sql_query(query12, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


With all the queries above. The culprit of this crime should be **"Jeremy Bowers"**